In [379]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow

In [380]:
#1
data=pd.read_excel("/content/Concrete_Data.xls")

In [381]:
data.shape

(1030, 9)

In [382]:
data.columns

Index(['Cement (component 1)(kg in a m^3 mixture)',
       'Blast Furnace Slag (component 2)(kg in a m^3 mixture)',
       'Fly Ash (component 3)(kg in a m^3 mixture)',
       'Water  (component 4)(kg in a m^3 mixture)',
       'Superplasticizer (component 5)(kg in a m^3 mixture)',
       'Coarse Aggregate  (component 6)(kg in a m^3 mixture)',
       'Fine Aggregate (component 7)(kg in a m^3 mixture)', 'Age (day)',
       'Concrete compressive strength(MPa, megapascals) '],
      dtype='object')

In [383]:
data.isnull().sum().sum()

0

In [384]:
#2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X= data.iloc[:,:-1].values
y= data.iloc[:,-1:].values
X=scaler.fit_transform(X)
y=scaler.fit_transform(y)
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.8,test_size=0.2)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(824, 8)
(206, 8)
(824, 1)
(206, 1)


In [385]:
#3
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import ReLU,LeakyReLU,ELU,PReLU





In [386]:

#in the problem statement ,it is stated to provide 10 inputs but according to the data ,only 8 can be provided in the best scenario.
classifier=Sequential()

classifier.add(Dense(units=10,activation='relu',input_dim =8))

classifier.add(Dense(units = 1, activation = 'linear'))

In [387]:
#4
from sklearn.metrics import r2_score
classifier.compile(optimizer="Adam",loss="mean_squared_error")
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 10)                90        
                                                                 
 dense_17 (Dense)            (None, 1)                 11        
                                                                 
Total params: 101 (404.00 Byte)
Trainable params: 101 (404.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [388]:
#5
model_history=classifier.fit(X_train,y_train,batch_size=10,epochs=100)

Epoch 1/100
83/83 [==============================] - 1s 2ms/step - loss: 0.9802
Epoch 2/100
83/83 [==============================] - 0s 1ms/step - loss: 0.6514
Epoch 3/100
83/83 [==============================] - 0s 2ms/step - loss: 0.5210
Epoch 4/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4537
Epoch 5/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4135
Epoch 6/100
83/83 [==============================] - 0s 2ms/step - loss: 0.3812
Epoch 7/100
83/83 [==============================] - 0s 2ms/step - loss: 0.3573
Epoch 8/100
83/83 [==============================] - 0s 2ms/step - loss: 0.3383
Epoch 9/100
83/83 [==============================] - 0s 2ms/step - loss: 0.3226
Epoch 10/100
83/83 [==============================] - 0s 2ms/step - loss: 0.3095
Epoch 11/100
83/83 [==============================] - 0s 2ms/step - loss: 0.2971
Epoch 12/100
83/83 [==============================] - 0s 2ms/step - loss: 0.2867
Epoch 13/100
83/83 [=================

In [389]:
#6 & #7
model_history.history.keys()


dict_keys(['loss'])

In [390]:
y_pred=classifier.predict(X_test)

7/7 [==============================] - 0s 1ms/step


In [391]:

mse=tensorflow.keras.losses.MeanSquaredError()
mse(y_test,y_pred).numpy()

0.13829808

In [392]:
#BONUS

In [393]:
#hyperparameter tuning with keras tuner

In [394]:
!pip install keras-tuner --upgrade

In [395]:
from kerastuner.tuners import RandomSearch

In [396]:
def build_model(hp):
  model=Sequential()
  model.add(Dense(units=10,activation='relu',input_dim=8))
  model.add(Dense(1,activation='linear'))
  optimizer=hp.Choice("optimizer",values=["adam","sgd","rmsprop","adadelta"])
  model.compile(optimizer=optimizer,loss='mean_squared_error')
  return model


In [397]:

tuner=RandomSearch(build_model,objective="val_loss",max_trials=5,directory='project1',project_name='model')

Reloading Tuner from project1/model/tuner0.json


In [410]:
tuner.search(X_train,y_train,validation_data=(X_test,y_test),epochs=5)

In [411]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [412]:
mymodel=tuner.get_best_models(num_models=1)[0]
mymodel.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                90        
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 101 (404.00 Byte)
Trainable params: 101 (404.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [413]:
from tensorflow.keras import optimizers
def build_model(hp):
    model = Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(Dense(units=hp.Int('units_' + str(i),
                                            min_value=1,
                                            max_value=8,
                                            step=32),
                               activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(
        optimizer=optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_squared_error')
    return model


In [414]:
tuner_second=RandomSearch(build_model,objective='val_loss',max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [415]:

tuner_second.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [416]:

tuner_second.get_best_hyperparameters()[0].values

{'optimizer': 'sgd',
 'num_layers': 2,
 'units_0': 1,
 'units_1': 1,
 'learning_rate': 0.01}

In [417]:
classifier2=tuner_second.get_best_models(num_models=1)[0]

In [418]:

classifier2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 9         
                                                                 
 dense_1 (Dense)             (None, 1)                 2         
                                                                 
 dense_2 (Dense)             (None, 1)                 2         
                                                                 
Total params: 13 (52.00 Byte)
Trainable params: 13 (52.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
